In [ ]:
# Import Necessary Packages
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix,precision_recall_curve,accuracy_score
from sklearn.metrics import auc,f1_score,roc_auc_score,roc_curve,recall_score
from sklearn.metrics import precision_score,recall_score,classification_report
from sklearn.ensemble import IsolationForest
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.models import Model, load_model
from keras.layers import Input, Dense
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras import regularizers

"""code for Meta Ensemble model-1 on credit card fraud data"""

# Load dataset
train = pd.read_csv('train.csv')
weight = pd.read_csv('weight.csv')
test = pd.read_csv('test.csv')
X = train.iloc[:,:-1]
Y = train.iloc[:,-1]
weightX = weight.iloc[:,:-1]
weightY = weight.iloc[:,-1]
testX = test.iloc[:,:-1]
testY = test.iloc[:,-1]

# Load autoencoder model
autoencoder = load_model('model.h5')
weight = autoencoder.predict(weightX)
weight = pd.DataFrame(weight)
train_A = autoencoder.predict(X)
train_A = pd.DataFrame(train_A)
test_A = autoencoder.predict(testX)
test_A = pd.DataFrame(test_A)
X_train,X_cross,Y_train,Y_cross = train_test_split(train_A,Y,
                                                   test_size=0.20,random_state=21, 
                                                   stratify=Y)

# Fine Tuning
Accuracy = []
f1 = []
for m in range(1,10):
    for n in range(50,101):
        clf = IsolationForest(n_estimators=n,max_samples="auto",
                              max_features=m,contamination=0.20,random_state=1)
        clf.fit(X_train)
        y_pred=clf.predict(X_cross)
        y_pred[y_pred==1]=0
        y_pred[y_pred==-1]=1
        f_1 = f1_score(Y_cross,y_pred)
        accuracy = accuracy_score(Y_cross,y_pred)
        f1.append(f_1)
        Accuracy.append(accuracy)
print("Best Accuracy for Isolation Forest is : ", max(Accuracy))
print("Best Accuracy for Isolation Forest parameters  grid no. : ", 
      Accuracy.index(max(Accuracy))+1)
print("Best f1 score for Isolation Forest is : ", max(f1))
print("Best f1 score for Isolation Forest parameters grid no. : ",
      f1.index(max(f1))+1)

# Training results
clf = IsolationForest(n_estimators=29,max_samples="auto",
                      max_features=3,contamination=0.024,random_state=1)
clf.fit(train_A)
y_pred=clf.predict(train_A)
y_pred[y_pred==1]=0
y_pred[y_pred==-1]=1
precision = precision_score(Y,y_pred)
recall = recall_score(Y,y_pred)
accuracy = accuracy_score(Y,y_pred)
print("Accuracy: ",accuracy)
print("Recall",recall)
print("Precision", precision)
print("Classification report : ")
print(classification_report(Y,y_pred))
print("Confusion Matrix : ")
print(confusion_matrix(Y,y_pred))

# Test Performance
y_pred=clf.predict(test_A)
y_pred[y_pred==1]=0
y_pred[y_pred==-1]=1
precision = precision_score(testY,y_pred)
recall = recall_score(testY,y_pred)
accuracy = accuracy_score(testY,y_pred)
print("Accuracy: ",accuracy)
print("Recall",recall)
print("Precision", precision)
print("Classification report : ")
print(classification_report(testY,y_pred))
print("Confusion Matrix : ")
print(confusion_matrix(testY,y_pred))
pd.DataFrame(y_pred, columns=['Class']).to_csv('AEif.csv',index = False)

# For Weight dataframe
y_pred=clf.predict(weightX)
y_pred[y_pred==1]=0
y_pred[y_pred==-1]=1
precision = precision_score(weightY,y_pred)
recall = recall_score(weightY,y_pred)
accuracy = accuracy_score(weightY,y_pred)
print("Accuracy: ",accuracy)
print("Recall",recall)
print("Precision", precision)
print("Classification report : ")
print(classification_report(weightY,y_pred))
print("Confusion Matrix : ")
print(confusion_matrix(weightY,y_pred))
pd.DataFrame(y_pred, columns=['Class']).to_csv('WAEif.csv',index = False)

"""code for Meta Ensemble model-2 on credit card fraud data"""

# Load data
train = pd.read_csv('train2.csv')
test = pd.read_csv('test2.csv')
X = train.iloc[:,:-1]
Y = train.iloc[:,-1]

testX = test.iloc[:,:-1]
testY = test.iloc[:,-1]
autoencoder = load_model('model.h5')
train_A = autoencoder.predict(X)
train_A = pd.DataFrame(train_A)
test_A = autoencoder.predict(testX)
test_A = pd.DataFrame(test_A)

X_train,X_cross,Y_train,Y_cross = train_test_split(train_A,Y,
                                                   test_size=0.20,
                                                   random_state=21,
                                                   stratify=Y)

# Fine Tuning
Accuracy = []
f1 = []
for m in range(1,10):
    for n in range(50,101):
        clf = IsolationForest(n_estimators=n,max_samples="auto",
                              max_features=m,contamination=0.20,
                              random_state=1)
        clf.fit(X_train)
        y_pred=clf.predict(X_cross)
        y_pred[y_pred==1]=0
        y_pred[y_pred==-1]=1
        f_1 = f1_score(Y_cross,y_pred)
        accuracy = accuracy_score(Y_cross,y_pred)
        f1.append(f_1)
        Accuracy.append(accuracy)
print("Best Accuracy for Isolation Forest is : ", max(Accuracy))
print("Best Accuracy for Isolation Forest parameters  grid no. : ", 
      Accuracy.index(max(Accuracy))+1)
print("Best f1 score for Isolation Forest is : ", max(f1))
print("Best f1 score for Isolation Forest parameters grid no. : ", 
      f1.index(max(f1))+1)

# Training Predictions
clf = IsolationForest(n_estimators=29,max_samples="auto",max_features=3,
                      contamination=0.024,random_state=1)
clf.fit(train_A)
y_pred=clf.predict(train_A)
y_pred[y_pred==1]=0
y_pred[y_pred==-1]=1
precision = precision_score(Y,y_pred)
recall = recall_score(Y,y_pred)
accuracy = accuracy_score(Y,y_pred)
print("Accuracy: ",accuracy)
print("Recall",recall)
print("Precision", precision)
print("Classification report : ")
print(classification_report(Y,y_pred))
print("Confusion Matrix : ")
print(confusion_matrix(Y,y_pred))
pd.DataFrame(y_pred, columns=['Class']).to_csv('tAEif2.csv',index = False)

# For test dataframe
y_pred=clf.predict(test_A)
y_pred[y_pred==1]=0
y_pred[y_pred==-1]=1
precision = precision_score(testY,y_pred)
recall = recall_score(testY,y_pred)
accuracy = accuracy_score(testY,y_pred)
print("Accuracy: ",accuracy)
print("Recall",recall)
print("Precision", precision)
print("Classification report : ")
print(classification_report(testY,y_pred))
print("Confusion Matrix : ")
print(confusion_matrix(testY,y_pred))
pd.DataFrame(y_pred, columns=['Class']).to_csv('AEif2.csv',index = False)

"""code for Meta Ensemble model-1 on The Abalone data"""

# Load Data
train = pd.read_csv("train.csv")
test =  pd.read_csv("test.csv")
weight1 =  pd.read_csv("weight1.csv")
X = train.iloc[:,:-1]
Y = train.iloc[:,-1]
X_test = test.iloc[:,:-1]
Y_test = test.iloc[:,-1]
weight1X = weight1.iloc[:,:-1]
weight1Y = weight1.iloc[:,-1]
autoencoder = load_model('model.h6')
weight1 = autoencoder.predict(weight1X)
weight1 = pd.DataFrame(weight1)
train_A = autoencoder.predict(X)
train_A = pd.DataFrame(train_A)
X_train,X_cross,Y_train,Y_cross = train_test_split(train_A,Y,
                                                   test_size=0.20,
                                                   random_state=21,
                                                   stratify=Y)
autoencoder = load_model('model.h6')
test_A = autoencoder.predict(X_test)
test_A = pd.DataFrame(test_A)

# Fine Tuning
Accuracy = []
f1 = []
for m in range(1,5):
    for n in range(90,101):
        clf = IsolationForest(n_estimators=n,max_samples="auto",
                              max_features=m,contamination=0.2,
                              random_state=1)
        clf.fit(X_train)
        y_pred=clf.predict(X_cross)
        y_pred[y_pred==1]=0
        y_pred[y_pred==-1]=1
        f_1 = f1_score(Y_cross,y_pred)
        accuracy = accuracy_score(Y_cross,y_pred)
        f1.append(f_1)
        Accuracy.append(accuracy)
print("Best Accuracy for Isolation Forest is : ", max(Accuracy))
print("Best Accuracy for Isolation Forest parameters  grid no. : ",
      Accuracy.index(max(Accuracy))+1)
print("Best f1 score for Isolation Forest is : ", max(f1))
print("Best f1 score for Isolation Forest parameters grid no. : ", 
      f1.index(max(f1))+10)

# Training results
y_pred=clf.predict(train_A)
y_pred[y_pred==1]=0
y_pred[y_pred==-1]=1
precision = precision_score(Y,y_pred)
recall = recall_score(Y,y_pred)
accuracy = accuracy_score(Y,y_pred)
print("Accuracy: ",accuracy)
print("Recall",recall)
print("Precision", precision)
print("Classification report : ")
print(classification_report(Y,y_pred))
print("Confusion Matrix : ")
print(confusion_matrix(Y,y_pred))
pd.DataFrame(y_pred, columns=['Class']).to_csv('AEif.csv',index = False)

# Test Results
clf = IsolationForest(n_estimators=99,max_samples="auto",
                      max_features=3,contamination=0.10,random_state=1)
clf.fit(train_A)
y_pred=clf.predict(test_A)
y_pred[y_pred==1]=0
y_pred[y_pred==-1]=1
precision = precision_score(Y_test,y_pred)
recall = recall_score(Y_test,y_pred)
accuracy = accuracy_score(Y_test,y_pred)
print("Accuracy: ",accuracy)
print("Recall",recall)
print("Precision", precision)
print("Classification report : ")
print(classification_report(Y_test,y_pred))
print("Confusion Matrix : ")
print(confusion_matrix(Y_test,y_pred))

# Weight Set Results
y_pred=clf.predict(weight1X)
y_pred[y_pred==1]=0
y_pred[y_pred==-1]=1
precision = precision_score(weight1Y,y_pred)
recall = recall_score(weight1Y,y_pred)
accuracy = accuracy_score(weight1Y,y_pred)
print("Accuracy: ",accuracy)
print("Recall",recall)
print("Precision", precision)
print("Classification report : ")
print(classification_report(weight1Y,y_pred))
print("Confusion Matrix : ")
print(confusion_matrix(weight1Y,y_pred))
pd.DataFrame(y_pred, columns=['Class']).to_csv('WAEisolation.csv',index = False)

"""code for Meta Ensemble model-2 on The Abalone data"""

# Load Data
train = pd.read_csv("train2.csv")
test =  pd.read_csv("test2.csv")
X = train.iloc[:,:-1]
Y = train.iloc[:,-1]
X_test = test.iloc[:,:-1]
Y_test = test.iloc[:,-1]
autoencoder = load_model('model.h6')
train_A = autoencoder.predict(X)
train_A = pd.DataFrame(train_A)
test_A = autoencoder.predict(X_test)
test_A = pd.DataFrame(test_A)

# Fine tuning
Accuracy = []
f1 = []
for m in range(1,5):
    for n in range(90,101):
        clf = IsolationForest(n_estimators=n,max_samples="auto",
                              max_features=m,contamination=0.2,random_state=1)
        clf.fit(X_train)
        y_pred=clf.predict(X_cross)
        y_pred[y_pred==1]=0
        y_pred[y_pred==-1]=1
        f_1 = f1_score(Y_cross,y_pred)
        accuracy = accuracy_score(Y_cross,y_pred)
        f1.append(f_1)
        Accuracy.append(accuracy)
print("Best Accuracy for Isolation Forest is : ", max(Accuracy))
print("Best Accuracy for Isolation Forest parameters  grid no. : ",
      Accuracy.index(max(Accuracy))+1)
print("Best f1 score for Isolation Forest is : ", max(f1))
print("Best f1 score for Isolation Forest parameters grid no. : ",
      f1.index(max(f1))+10)

# Training Results
print("Accuracy: ",accuracy_score(Y,y_pred1))
print("Recall",recall)
print("Precision", precision)
print("Classification report : ")
print(classification_report(Y,y_pred1))
print("Confusion Matrix : ")
print(confusion_matrix(Y,y_pred1))
pd.DataFrame(y_pred1, columns=['Class']).to_csv('tAEif2.csv',index = False)

# Test Results
clf = IsolationForest(n_estimators=99,max_samples="auto",
                      max_features=3,contamination=0.10,random_state=1)
clf.fit(train_A)
y_pred=clf.predict(test_A)
y_pred1 = clf.predict(train_A)
y_pred[y_pred==1]=0
y_pred[y_pred==-1]=1
y_pred1[y_pred1==1]=0
y_pred1[y_pred1==-1]=1
precision = precision_score(Y_test,y_pred)
recall = recall_score(Y_test,y_pred)
accuracy = accuracy_score(Y_test,y_pred)
print("Accuracy: ",accuracy)
print("Recall",recall)
print("Precision", precision)
print("Classification report : ")
print(classification_report(Y_test,y_pred))
print("Confusion Matrix : ")
print(confusion_matrix(Y_test,y_pred))
pd.DataFrame(y_pred, columns=['Class']).to_csv('AEif2.csv',index = False)